In [47]:
!source ~/.bash_profile
%env PGDATABASE=postgres
%env PGHOST=10.128.0.2
%env PGPORT=5432
%env PGUSER=postgres
%env PGPASSWORD=JordanGrahamRepo
%env PGINSTANCE=pg

env: PGDATABASE=postgres
env: PGHOST=10.128.0.2
env: PGPORT=5432
env: PGUSER=postgres
env: PGPASSWORD=JordanGrahamRepo
env: PGINSTANCE=pg


In [45]:
%env BASE_DIR=/home/jupyter/JordanGrahamRepo

env: BASE_DIR=/home/jupyter/JordanGrahamRepo


In [3]:
#Creating pagila schema and objects

In [4]:
!gsutil cp gs://cs327e-open-access/load_testing/us-500.zip $BASE_DIR

Copying gs://cs327e-open-access/load_testing/us-500.zip...
/ [1 files][ 37.1 KiB/ 37.1 KiB]                                                
Operation completed over 1 objects/37.1 KiB.                                     


In [5]:
!gsutil cp gs://cs327e-open-access/load_testing/us-1000000.zip $BASE_DIR

Copying gs://cs327e-open-access/load_testing/us-1000000.zip...
\ [1 files][ 69.7 MiB/ 69.7 MiB]                                                
Operation completed over 1 objects/69.7 MiB.                                     


In [6]:
!unzip $BASE_DIR/us-500.zip

Archive:  /home/jupyter/JordanGrahamRepo/us-500.zip
  inflating: us-500.csv              


In [7]:
!unzip $BASE_DIR/us-1000000.zip

Archive:  /home/jupyter/JordanGrahamRepo/us-1000000.zip
  inflating: us-1000000.csv          


In [8]:
import csv

max_dict = {
    'first_name': 0,
    'last_name': 0,
    'company_name': 0,
    'address': 0,
    'city': 0,
    'county': 0,
    'state': 0,
    'zip': 0,
    'phone1': 0,
    'phone2': 0,
    'email': 0,
    'web': 0
}

with open('us-500.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=',')
    for row in csv_reader:
        for i in range(len(row)):
            if max_dict[list(max_dict)[i]] < len(row[i]):
                max_dict[list(max_dict)[i]] = len(row[i])
max_dict

{'first_name': 10,
 'last_name': 13,
 'company_name': 30,
 'address': 31,
 'city': 19,
 'county': 20,
 'state': 5,
 'zip': 5,
 'phone1': 12,
 'phone2': 12,
 'email': 34,
 'web': 42}

In [50]:
!psql -f $BASE_DIR/create_table.sql

psql:/home/jupyter/JordanGrahamRepo/create_table.sql:1: NOTICE:  drop cascades to table load_testing.person
DROP SCHEMA
CREATE SCHEMA
SET
CREATE TABLE


In [10]:
pip install psycopg[binary]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 56.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [51]:
%%timeit
import psycopg
import csv

host = '10.128.0.2'
password = 'JordanGrahamRepo'
url = "dbname='postgres' user='postgres' host={} port=5432 password={}".format(host, password)

conn = psycopg.connect(url)

print(conn)

sql = "INSERT INTO load_testing.Person (first_name, last_name, company_name, address, city, county, state, zip, phone1,\
       phone2, email, web) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

try:
    with open('us-1000000.csv', newline='') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=',')
        columns = next(csv_reader)
        count = 0
        with conn.cursor() as cur:
            for row in csv_reader:
                cur.execute(sql, row)
                count += 1
                if count % 5000 == 0:
                    conn.commit()
                    print(count, "records inserted successfully into Person table")

except psycopg.errors.OperationalError as error:
    print("Failed to insert record into Person table {}".format(error))
                
finally:
    conn.close()
    print("Postgres connection is closed")


<psycopg.Connection [IDLE] (host=10.128.0.2 database=postgres) at 0x7f5ce8a39310>
5000 records inserted successfully into Person table
10000 records inserted successfully into Person table
15000 records inserted successfully into Person table
20000 records inserted successfully into Person table
25000 records inserted successfully into Person table
30000 records inserted successfully into Person table
35000 records inserted successfully into Person table
40000 records inserted successfully into Person table
45000 records inserted successfully into Person table
50000 records inserted successfully into Person table
55000 records inserted successfully into Person table
60000 records inserted successfully into Person table
65000 records inserted successfully into Person table
70000 records inserted successfully into Person table
75000 records inserted successfully into Person table
80000 records inserted successfully into Person table
85000 records inserted successfully into Person table
9

In [52]:
#Count
!psql -c "select count(*) from load_testing.Person"

  count  
---------
 8000000
(1 row)

